In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)

In [3]:
import sys
sys.path.insert(0, "../code-previous")
sys.path.insert(0, "../tagger-scrap")

sys.path.insert(0, "..")

In [4]:
import os
from nltk.tag.stanford import StanfordPOSTagger
java_path = "/home/sreekumar_s/jdk1.8.0_131/bin/java"
os.environ['JAVAHOME'] = java_path


import CMUTweetTagger

In [5]:
path_to_model = "../pos-tagger-models/english-bidirectional-distsim.tagger"
path_to_jar = "../pos-tagger-models/stanford-postagger.jar"
tagger = StanfordPOSTagger(path_to_model, path_to_jar)
tagger.java_options='-mx4096m'

In [6]:
import pandas as pd
import numpy as np
import nltk
stopwords = nltk.corpus.stopwords.words('english')

import plotly

import swifter
import dask.dataframe as dd
from dask.multiprocessing import get

from unicodedata import category
from tqdm import tqdm
tqdm.pandas(desc="Progress")

import seaborn as sns
sns.set()
import mutual_information
import matplotlib.pyplot as plt
path2rawdata = '/mnt/disks/vault/wos2017-parsed/'

In [7]:
raw_data_path = "/mnt/disks/vault/analysis-data/nature-science-data-full/nature_science_journal_data.pql"
data = pd.read_pickle(raw_data_path)

data = data[(data.Journal == "NATURE") & (data.PubYear_x > 1980) & (data.PubYear_x < 1991)]

In [8]:
data.PubYear_x.unique()

array([1981, 1982, 1987, 1983, 1988, 1984, 1989, 1990, 1985, 1986])

In [9]:
len(data)

16739

In [10]:
data.head()

,ArticleID,PubYear_x,Doctypes,Title,Journal,Journal Abbreviation ISO,PubYear_y,c5
0,A1981MB41800038,1981,Article,INDEPENDENT CONTROL OF IMMUNOGLOBULIN HEAVY AND LIGHT CHAIN EXPRESSION IN A MURINE PRE-B-CELL LINE,NATURE,Nature,1981,23.0
1,A1981MK47400052,1981,Article,DEMETHYLATION OF CPG SITES IN DNA OF EARLY RABBIT TROPHOBLAST,NATURE,Nature,1981,14.0
5,000204440900038,1981,Article,Comb-layering in carbonatite dykes,NATURE,Nature,1981,5.0
8,A1981MT99400035,1981,Article,STRUCTURE SENSITIVITY IN THE IRON SINGLE-CRYSTAL CATALYZED SYNTHESIS OF AMMONIA,NATURE,Nature,1981,6.0
9,A1981LS25500015,1981,Article,IR VARIABILITY OF SS433,NATURE,Nature,1981,4.0


__Add tagging from the two parsers__

In [11]:
data['stanford_tagger_tags'] = data['Title'].swifter.apply(lambda x: tagger.tag(x.split()))

Pandas Apply:  18%|█▊        | 3000/16739 [1:22:26<5:14:56,  1.38s/it] IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Pandas Apply: 100%|██████████| 16739/16739 [7:40:29<00:00,  1.29s/it]  


In [12]:
import pickle
data.to_pickle("/mnt/disks/vault/analysis-data/stanford_tagger_data/pos_tagged_stanford_1980_1990.pql")

In [ ]:
#ark_tagger_outputs = CMUTweetTagger.runtagger_parse(list(data['Title']))
#data['ark_tagger_tags'] = pd.Series(ark_tagger_outputs)

In [ ]:
## https://stackoverflow.com/questions/45545110/how-do-you-parallelize-apply-on-pandas-dataframes-making-use-of-all-cores-on-o

ddata = dd.from_pandas(data, npartitions=30)

res = ddata.map_partitions(lambda df: df['Title'].apply((lambda x: tag(x.split())), axis=1)).compute(get=get)